In [1]:
from pyspark.sql import SparkSession
import json

In [2]:
spark = SparkSession.builder\
    .appName("Pretrained Data Pipeline")\
    .master("local[*]")\
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/04 03:18:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
src_files = "/harddisk/user/keminglu/pretrained_data_processed/wikipedia_with_mention_wo_title_simplified_aug/corpus"
raw_data = sc.textFile(src_files)

In [4]:
data = raw_data.map(json.loads)#.filter(lambda x: x['aug_type'] == 'aug_default')

In [5]:
data.take(1)

[{'id': '26037682',
  'title': 'Harlan Hatcher',
  'inputs': 'Harlan Henthorne Hatcher (September 9, 1898 – February 25, 1998) served as the eighth President of the University of Michigan from 1951 to 1967.',
  'targets': '{"entities": [{"mention": "University of Michigan", "title": "University of Michigan", "description": "public research university in Ann Arbor, Michigan, United States", "aliases": ["UMich", "UM", "U of Michigan", "U of M", "Univ of Michigan"]}], "triplets": []}',
  'prompt': 'Extract entities.',
  'aug_type': 'aug_default',
  'aug_info': None}]

In [81]:
# Mention
total_mention = data.flatMap(lambda x: [each['mention'] for each in json.loads(x['targets'])['entities']]).count()
total_mention

109411

In [82]:
print(data.flatMap(lambda x: [each['mention'] for each in json.loads(x['targets'])['entities'] if each['ood'] == 'in']).count())
print(data.flatMap(lambda x: [each['mention'] for each in json.loads(x['targets'])['entities'] if each['ood'] != 'in']).count())

79799
29612


In [83]:
# Article
data.count()

24393

In [84]:
# Entities
data.flatMap(lambda x: [each['title'] for each in json.loads(x['targets'])['entities']]).distinct().count()

39086

In [7]:
train_entities = data.flatMap(lambda x: [each['title'] for each in json.loads(x['targets'])['entities']]).distinct().collect()

In [71]:
print(data.flatMap(lambda x: [each['title'] for each in json.loads(x['targets'])['entities'] if each['ood'] == 'in']).distinct().count())
print(data.flatMap(lambda x: [each['title'] for each in json.loads(x['targets'])['entities'] if each['ood'] != 'in']).distinct().count())

29612
9474


In [72]:
# Aliases
data.flatMap(lambda x: sum([each['aliases'] for each in json.loads(x['targets'])['entities'] if 'aliases' in each], [])).distinct().count()

37809

In [ ]:
# Types
data.flatMap(lambda x: sum([each['type'] for each in json.loads(x['targets'])['entities'] if 'type' in each], [])).distinct().count()

Exception in thread "serve RDD 210" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [89]:
# Triplets
data.flatMap(lambda x: json.loads(x['targets'])['triplets']).count()

45758

In [75]:
# Relations
data.flatMap(lambda x: sum([[r for r in each['relations']] for each in json.loads(x['targets'])['triplets']], [])).distinct().count()

398

In [86]:
# Description
data.flatMap(lambda x: [1 if 'description' in each else 0 for each in json.loads(x['targets'])['entities']]).sum() / total_mention

0.9268172304430085

In [87]:
data.flatMap(lambda x: [1 if 'aliases' in each else 0 for each in json.loads(x['targets'])['entities']]).sum() / total_mention

0.5257880834650995

In [88]:
data.flatMap(lambda x: [1 if 'type' in each else 0 for each in json.loads(x['targets'])['entities']]).sum() / total_mention

0.7069216075166117

# Out-of-ontology type and rel

In [96]:
train_types = data.flatMap(lambda x: sum([each['type'] for each in json.loads(x['targets'])['entities'] if 'type' in each], [])).distinct().collect()

In [97]:
train_rel = data.flatMap(lambda x: sum([[r for r in each['relations']] for each in json.loads(x['targets'])['triplets']], [])).distinct().collect()

In [6]:
train_triplets = data.flatMap(lambda x: sum([[(each['head'], each['tail'], r) for r in each['relations']] for each in json.loads(x['targets'])['triplets']], [])).distinct().collect()

In [8]:
train_triplets[10]

('Daniel Levy', 'Speaker', 'position held')

In [100]:
with open("/harddisk/user/keminglu/pretrained_data_processed/wikipedia_with_mention_wo_title_simplified_aug/train_types.json", "w") as f:
    json.dump(train_types, f)

In [101]:
with open("/harddisk/user/keminglu/pretrained_data_processed/wikipedia_with_mention_wo_title_simplified_aug/train_relations.json", "w") as f:
    json.dump(train_rel, f)

In [9]:
with open("/harddisk/user/keminglu/pretrained_data_processed/wikipedia_with_mention_wo_title_simplified_aug/train_triplets.json", "w") as f:
    json.dump(train_triplets, f)

In [9]:
with open("/harddisk/user/keminglu/pretrained_data_processed/wikipedia_with_mention_wo_title_simplified_aug/train_entities.json", "w") as f:
    json.dump(train_entities, f)